# Extração de contas dos programas do FNDE

Importando os pacotes necessários

In [ ]:
import urllib.request

In [ ]:
import re

In [ ]:
import pandas as pd

In [ ]:
import ssl

Desabilitando a verificação do contexto

In [ ]:
context = ssl._create_unverified_context()

Criando a lista para armazenar os anos da pesquisa

In [ ]:
anos = []

Definindo o ano inicial da pesquisa

In [ ]:
primeiro_ano = 2018

Definindo o ano final da pesquisa

In [ ]:
ultimo_ano = 2018

Carregando a lista com os anos a serem pesquisados

In [ ]:
for ano in range(primeiro_ano, ultimo_ano + 1):
    anos.append(ano)

Importando o banco de dados com os códigos e os nomes dos programas a serem pesquisados

In [ ]:
programas = pd.read_excel('programas.xlsx')

Separando os códigos do banco de dados

In [ ]:
codigos = programas['Código']

Definindo o formato da string da url da pesquisa

In [ ]:
url = 'https://www.fnde.gov.br/sigefweb/index.php/extratos?ano={}&programa={}&confirmar=Confirmar'

Criando uma lista para armazenar as urls da pesquisa

In [ ]:
urls = []

Carregando a lista com as urls a serem pesquisadas

In [ ]:
for ano in reversed(anos):
    for codigo in codigos.iteritems():
        urls.append(url.format(ano, codigo[1]))

Conferindo as urls carregadas na lista

In [ ]:
urls

Criando uma lista para carregar as esferas pesquisadas em cada url

In [ ]:
lista_esferas = []

Carregando a lista de esferas a partir das urls

In [ ]:
for site in urls:
    with urllib.request.urlopen(site, context=context) as response:
       html = response.read().decode('ansi')
    esferas = re.findall(r'</a></td><td><a href=\"(.*?)\">', html)
    lista_esferas = lista_esferas + esferas

Conferindo a lista de esferas carregada

In [ ]:
lista_esferas

Definindo a url do site do FNDE

In [ ]:
site_fnde = 'https://www.fnde.gov.br'

Criando a lista das urls das entidades de cada programa

In [ ]:
lista_entidades = []

Carregando a lista de entidades de cada programa

In [ ]:
for esfera in lista_esferas:
    with urllib.request.urlopen(site_fnde + esfera, context=context) as response:
       html = response.read().decode('ansi')
    entidades = re.findall(r'<tr><td><a href=\"(.*?)\">', html)
    lista_entidades = lista_entidades + entidades

Criando as listas onde serão armazenadas as informações das entidades pesquisadas

In [ ]:
lista_cnpjs = []

In [ ]:
lista_razoes = []

In [ ]:
lista_bancos = []

In [ ]:
lista_contas = []

In [ ]:
lista_programas = []

Carregando as listas com as informações de cada entidade. As expressões regulares foram construídas com a ferramenta disponível no site: http://www.regexlab.com/wild2regex

In [ ]:
for entidade in lista_entidades:    
    with urllib.request.urlopen(site_fnde + entidade, context=context) as response:
        html = response.read().decode('ansi')
    cnpj = re.findall(r'<th>CNPJ\:</th>\s+<td>((?:(?!</td>)(?:.|\n))*)</td>', html, re.MULTILINE)
    lista_cnpjs.append(cnpj)
    razao = re.findall(r'<th>Razão\s+Social\:</th>\s+<td>((?:(?!</td>)(?:.|\n))*)</td>', html, re.MULTILINE)
    lista_razoes.append(razao)
    banco = re.findall(r'<th>Banco\:</th>\s+<td>((?:(?!</td>)(?:.|\n))*)</td>', html, re.MULTILINE)
    lista_bancos.append(banco)
    conta = re.findall(r'<th>Conta\s+Corrente\:</th>\s+<td>((?:(?!</td>)(?:.|\n))*)</td>', html, re.MULTILINE)
    lista_contas.append(conta)
    programa = re.findall(r'<th>Programa\:</th>\s+<td>((?:(?!</td>)(?:.|\n))*)</td>', html, re.MULTILINE)
    lista_programas.append(programa)

Conferindo o conteúdo das listas das informações

In [ ]:
len(lista_cnpjs)

In [ ]:
len(lista_razoes)

In [ ]:
len(lista_bancos)

In [ ]:
len(lista_contas)

In [ ]:
len(lista_programas)

Criando um dicionário com as informações pesquisadas

In [ ]:
entidades = ['CNPJ': lista_cnpjs, 'RAZÃO SOCIAL': lista_razoes, 'BANCO': lista_bancos, 'CONTA': lista_contas, 'PROGRAMA': lista_programas]

Criando um novo banco de dados a partir do dicionário entidades

In [ ]:
df = pd.DataFrame.from_dict(entidades)